In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_total = pd.read_csv('cases-brazil-states.csv')
df_total.head()

In [ ]:
df = df_total[df_total['estado'] != 'TOTAL']
df['data'] = pd.to_datetime(df_total['data']).dt.date 
df.head()

In [ ]:
plt.figure(figsize=(20,8))
fig = sns.lineplot(x= 'data', y='Obitos', hue ='estado', data=df)
fig.set_title('òbitos por estado em função do tempo', loc ='left', fontsize = 24)
fig.set_xlabel('Data', fontsize = 24)
fig.set_ylabel('Óbitos', fontsize = 24)

In [ ]:
df_sp = df[df['estado']=='SP']
df_sp.head()

In [ ]:
plt.figure(figsize=(20,8))
fig = sns.lineplot(x= 'data', y='Obitos', data=df_sp)
fig.set_title('òbitos por estado em função do tempo SP', loc ='left', fontsize = 24)
fig.set_xlabel('Data', fontsize = 24)
fig.set_ylabel('Log Óbitos', fontsize = 24)
fig.set_yscale('log')

In [ ]:
plt.figure(figsize=(20,8))
fig = sns.lineplot(x= 'data', y='Obitos',hue ='estado', data=df)
fig.set_title('òbitos por estado em função do tempo ', loc ='left', fontsize = 24)
fig.set_xlabel('Data', fontsize = 24)
fig.set_ylabel('Log Óbitos', fontsize = 24)
fig.set_yscale('log')

In [ ]:
plt.figure(figsize=(20,8))
fig = sns.barplot(x='data',y='Obitos',hue='estado',data=df)
fig.set_title('Óbitos por estado em função do tempo', loc='left',fontsize=24)
fig.set_xlabel('Data',fontsize=24)
fig.set_ylabel('Óbitos',fontsize=24)

In [ ]:
df_sp.head()

In [ ]:
plt.figure(figsize=(20,8))
fig = plt.bar(df_sp.data, df_sp.suspeitos, label = 'Suspeitos')
plt.bar(df_sp.data, df_sp.recuperados, label = 'Recuperados')
plt.bar(df_sp.data, df_sp.Obitos, label = 'Obitos')
plt.xlabel('Data', fontsize =24)
plt.ylabel('Obitos, Recuperados e Ssupeitos SP', fontsize =24, rotation=90)
plt.legend()


In [ ]:
def plotar_linha(titulo,xlabel,ylabel,x,y,dataset,hue):
  ax =sns.lineplot(x=x,y=y, hue=hue,data=dataset)
  ax.figure.set_size_inches(12,6)
  ax.set_title(titulo, loc ='left', fontsize =20)
  ax.set_xlabel(xlabel, fontsize = 20)
  ax.set_ylabel(ylabel, fontsize = 20, rotation = 90)

In [ ]:
def plotar_barra(titulo,xlabel,ylabel,x,y,dataset):
  plt.figure(figsize=(20,6))
  ax= plt.bar(dataset[x], dataset[y])
  plt.title(titulo, loc = 'left', fontsize =20)
  plt.xlabel(xlabel, fontsize=20)
  plt.ylabel(ylabel, fontsize = 20, rotation =90)

In [ ]:
plotar_barra('Novos casos SP', "Data", 'Novos casos', 'data', 'novosCasos', df_sp)

In [ ]:
df_sp['aceleracaoCasos'] = df_sp['novosCasos'].diff()
df_sp['aceleracaoObitos'] = df_sp['novosObitos'].diff()
df_sp.head()

In [ ]:
plotar_barra('Aceleração de Òbitos SP', 'Data', 'Aceleração' , 'data', 'aceleracaoObitos', df_sp )

In [ ]:
df_sp['mediaObitos'] = df_sp.novosObitos.rolling(window= 7, center =False).mean()
df_sp['mediaObitos_14'] = df_sp.novosObitos.rolling(window= 14, center =False).mean()
df_sp.head(14)

In [ ]:
plotar_linha('Media móvel de óbitos SP', 'Data', 'Média', 'data', 'mediaObitos', df_sp, None)
plotar_linha('Media móvel de óbitos SP', 'Data', 'Média', 'data', 'mediaObitos_14', df_sp, None)
plt.bar(df_sp['data'], df_sp['novosObitos'], color = 'lightgrey')


In [ ]:
from pandas.plotting import autocorrelation_plot

In [ ]:
ax = plt.figure(figsize=(12,6))
ax.suptitle('Correlação de Òbitos', fontsize =18, x=0.26, y= 0.95) 
autocorrelation_plot(df_sp['Obitos'])
ax=ax

In [ ]:
ax = plt.figure(figsize=(12,6))
ax.suptitle('Correlação dos novos Obitos', fontsize =18, x=0.26, y= 0.95) 
autocorrelation_plot(df_sp['novosObitos'])
ax=ax

In [ ]:
ax = plt.figure(figsize=(12,6))
ax.suptitle('Correlação dos novos Obitos média móvel', fontsize =18, x=0.26, y= 0.95) 
autocorrelation_plot(df_sp['mediaObitos'][7:])
ax=ax

In [ ]:
ax = plt.figure(figsize=(12,6))
ax.suptitle('Autocorrelação Òbitos', fontsize =18, x=0.26, y= 0.95) 
autocorrelation_plot(df_sp['aceleracaoObitos'][1:])
ax=ax

In [ ]:
df_sp['mediaAccObitos'] = df_sp['aceleracaoObitos'][1:].rolling(window=7,center=False).mean()

In [ ]:
ax = plt.figure(figsize=(12,6))
ax.suptitle('Autocorrelação da média da aceleração de óbitos',fontsize=18,x=0.26,y=0.95)
autocorrelation_plot(df_sp['mediaAccObitos'][7:])
ax=ax

In [ ]:
df_sp['mes'] = pd.to_datetime(df_sp['data'])
df_sp['dia_da_semana'] = df_sp['mes'].dt.day_name()
df_sp.head() 

In [ ]:
dias_traduzidos = {'Monday': 'Segunda', 'Tuesday' : 'Terça', 'Wednesday':'Quarta', 'Thursday':'Quinta', 'Friday':'Sexta', 'Saturday':'Sábado', 'Sunday':'Domingo'}

In [ ]:
df_sp['dia_da_semana'] = df_sp['dia_da_semana'].map(dias_traduzidos)
df_sp.head()

In [ ]:
agrupados_sp = df_sp.groupby('dia_da_semana')['Obitos','novosObitos','aceleracaoObitos'].mean().round()

In [ ]:
agrupados_sp['dia_da_semana'] = agrupados_sp.index
agrupados_sp.head(7)

In [ ]:
agrupados_sp = agrupados_sp.reindex(['Domingo', 'Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sabado'])

In [ ]:
sns.barplot(agrupados_sp['dia_da_semana'], agrupados_sp.novosObitos)

In [ ]:
def criar_features(df,label=None):
    df = df.copy()
    df['diasemana'] = df['mes'].dt.dayofweek
    df['mês'] = df['mes'].dt.month
    df['diamês'] = df['mes'].dt.day
    df['semana'] = df['mes'].dt.weekofyear

    X = df[['diasemana','mês','diamês','semana']]

    if label:
        y = df[label]
        return X,y
    return X

In [ ]:
plt.figure(figsize=(20,8))
g = sns.lineplot(x='semana',y='Obitos',data=df)

g.axes.set_xlabel('Semana do ano',fontsize=15)
g.axes.set_ylabel('Óbitos',fontsize=15)

plt.suptitle('Óbitos por semana',fontsize=15)